In [ ]:
% matplotlib inline
% load_ext autoreload
% autoreload 2

In [ ]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
from deep_visual_odometry.kitti_utils import KITTIdata
from matplotlib import pyplot as plt

In [ ]:
basedir = 'dataset'

In [ ]:
from deep_visual_odometry.odometry_model import OdomModel

In [ ]:
batch_size = 256         # Sequences per batch
num_steps = 8         # Number of sequence steps per batch
rnn_size = 64          # Size of hidden layers in rnn_cell
num_layers = 2           # Number of hidden layers
learning_rate = 0.001

In [ ]:
kitti_data = KITTIdata(basedir, 
                       sequences = ['00','01'],
                       sequence_len = num_steps,
                       val_frac = 0,
                       test_frac = 0.1,
                       img_size=(150,50))

In [ ]:
velocities_total = []
poses_total = []
for sequence in kitti_data.sequences:
    velocities_total.append(kitti_data.velocities[sequence])
    poses_total.append(kitti_data.poses[sequence])

In [ ]:
velocities_total = np.vstack(velocities_total)
poses_total = np.vstack(poses_total)
mean_velo = np.mean(velocities_total,axis = 0)
std_velo = np.std(velocities_total,axis = 0)
mean_pose = np.mean(poses_total,axis = 0)
std_pose = np.std(poses_total,axis = 0)

In [ ]:
for sequence in kitti_data.sequences:
    kitti_data.poses[sequence] = kitti_data.poses[sequence]-mean_pose
    kitti_data.poses[sequence] /= std_pose
    
    kitti_data.velocities[sequence] = kitti_data.velocities[sequence]-mean_velo
    kitti_data.velocities[sequence] /= std_velo

In [ ]:
model = OdomModel(batch_size = 256, 
                  num_steps = 8,
                  cell_type = 'LSTM',
                  rnn_size = 128,
                  num_layers = 2,
                  learning_rate = 0.001)
model.train(kitti_data, max_count = 10000, save_every_n = 1000)

In [ ]:
_, test_velocities,test_poses = kitti_data.load_data_test()
print(test_velocities.shape,test_poses.shape)

In [ ]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
predictions = model.test(checkpoint,test_velocities,256)